# ETL Pipeline and Data Modeling with Apache Cassandra

## Step 1. Extract event data and Transform relevant data to a single csv file

In [1]:
# Import Python packages 
import cassandra
import os
import glob
import csv

#### Create a list of filepaths to process original event csv data files

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # skip hidden files and directories
    if '/.' in root:
        continue
    dirs = [d for d in dirs if not d.startswith('.')]

    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Process the files to create a csv file with relevant data that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


## Step 2. Load data into the Apache Cassandra database. 

#### The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">


### Setup & Initialize the Apache Cassandra Database

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # Establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_cassandra')
except Exception as e:
    print(e)



### Model the Apache Cassandra database tables based on the desired queries.

#### Query 1. Show the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [7]:
## Create a playlist_history table with primary key (session_id, item_in_session)
query = "CREATE TABLE IF NOT EXISTS playlist_history "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, duration decimal, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [8]:
# read the relevant data from CSV file and populate the Apache Cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT query to store desired data
        query = "INSERT INTO playlist_history (session_id, item_in_session, artist_name, song_title, duration)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        
        ## CSV column element is mapped to the matching data column and inserted into the table.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

##### Query 1 Results:

In [9]:
## Ensure the query produces the desired results
query = "select artist_name, song_title, duration from playlist_history WHERE SESSION_ID=338 AND ITEM_IN_SESSION=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,',', row.song_title,',', row.duration)

Faithless , Music Matters (Mark Knight Dub) , 495.3073


#### Query 2. Show only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
# Create a user_playlist_history table with primary key (user_id, session_id, item_in_session)
query = "CREATE TABLE IF NOT EXISTS user_playlist_history "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_firstname text, user_lastname text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [11]:
# read the relevant data from CSV file and populate the Apache Cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT query to store desired data
        query = "INSERT INTO user_playlist_history (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        
        ## CSV column element is mapped to the matching data column and inserted into the table.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

##### Query 2 Results:

In [12]:
## Ensure the query produces the desired results
query = "select song_title, artist_name, user_firstname, user_lastname from user_playlist_history WHERE USER_ID=10 AND SESSION_ID=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title,',', row.artist_name,',', row.user_firstname, row.user_lastname)

Keep On Keepin' On , Down To The Bone , Sylvie Cruz
Greece 2000 , Three Drives , Sylvie Cruz
Kilometer , Sebastien Tellier , Sylvie Cruz
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Lonnie Gordon , Sylvie Cruz


#### Query 3. Show every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [13]:
# Create a song_user_history table with primary key (song_title, user_id, session_id, item_in_session)
query = "CREATE TABLE IF NOT EXISTS song_user_history "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_firstname text, user_lastname text, PRIMARY KEY (song_title, user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [14]:
# read the relevant data from CSV file and populate the Apache Cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT query to store desired data
        query = "INSERT INTO song_user_history (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        
        ## CSV column element is mapped to the matching data column and inserted into the table.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

##### Query 3 Results:

In [15]:
## Ensure the query produces the desired results
query = "select song_title, user_firstname, user_lastname from song_user_history WHERE SONG_TITLE='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title,',', row.user_firstname, row.user_lastname)

All Hands Against His Own , Jacqueline Lynch
All Hands Against His Own , Tegan Levine
All Hands Against His Own , Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
query = "DROP TABLE IF EXISTS playlist_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_playlist_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()